- 模型应用
  - 模型加载
  - 模型测试集评估
    - 准确率
    - 召回率
    - F1


In [1]:
# 测试集评论数据测试

import time

import torch
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import dataset
import embedding
import model
from init import config

stopwords = dataset.load_stopwords(config.stopwords_path)
model = model.SentimentAnalysisModel().load(config.model_pt_path).to(config.device)
print('test_model[{:s}]'.format(config.model_pt_path), )
word_embedding = embedding.WordEmbedding().load(config.embedding_pt_path).to(config.device)
print('word_embedding[{:s}]'.format(config.embedding_pt_path), )
test_contents, test_emotions = dataset.load_contents_labels(config.test_data_path)
test_tokens = dataset.tokenization(test_contents, stopwords)
test_dataset = dataset.SentimentAnalysisDataset(tokens=test_tokens, emotions=test_emotions,
                                                sequence_length=config.model_sequence_length,
                                                key_to_index=word_embedding.key_to_index, ).dataset
loss_fn = torch.nn.CrossEntropyLoss()
model.eval()
data, targets = test_dataset.tensors[0].to(config.device), test_dataset.tensors[1].to(config.device)
output = model(data)
test_loss = loss_fn(output, targets).item()
predicted = output.argmax(dim=1).cpu().numpy()
labels = targets.cpu().numpy()
print("每个类别的精确率和召回率：\n",
      classification_report(labels, predicted, target_names=dataset.emotion2label.keys()))

test_macro_precision = precision_score(labels, predicted, average='macro', zero_division=0)  # 宏精准率
test_macro_recall = recall_score(labels, predicted, average='macro', zero_division=0)  # 宏召回率
test_macro_f1 = f1_score(labels, predicted, average='macro')  # 宏F1值
print('test_loss[{:f}]'.format(test_loss),
      'test_macro_precision[{:f}]'.format(test_macro_precision),
      'test_macro_recall[{:f}]'.format(test_macro_recall),
      'test_macro_f1[{:f}]'.format(test_macro_f1),
      time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

/Users/yangyunxiang/Documents/project/HAI/DL/deep_learning_2024/embedding.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename)
Building pre

test_model[pt/model_Transformer_512_0.4_ReLU_100_35_256_1_0.001_0.8_pt]
word_embedding[pt/embedding.pt]


Loading model cost 0.347 seconds.
DEBUG:jieba:Loading model cost 0.347 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


每个类别的精确率和召回率：
               precision    recall  f1-score   support

        非常满意       0.76      0.54      0.63       885
          满意       0.66      0.65      0.66      1247
          中立       0.31      0.40      0.35       267
         不满意       0.54      0.59      0.56      1150
       非常不满意       0.72      0.76      0.74      1320

    accuracy                           0.63      4869
   macro avg       0.60      0.59      0.59      4869
weighted avg       0.65      0.63      0.64      4869

test_loss[0.952515] test_macro_precision[0.598000] test_macro_recall[0.589629] test_macro_f1[0.588184] 2024-10-29 17:38:39


In [4]:
# 随机评论数据测试
import numpy
import torch
import model
import dataset
import embedding
from init import config

stopwords = dataset.load_stopwords(config.stopwords_path)
model = model.SentimentAnalysisModel().load(config.model_pt_path).to(config.device)
print('test_model[{:s}]'.format(config.model_pt_path), )
word_embedding = embedding.WordEmbedding().load(config.embedding_pt_path).to(config.device)
print('word_embedding[{:s}]'.format(config.embedding_pt_path), )

contents = ["感觉这个秤非常小巧精致实用。自从看到家人买来这个秤以后，其他的包括秤（包括金器秤）都被闲置了",
            "第三次购买了，还算挺好的，希望能更耐用一些。",
            "没什想说的，中规中矩，没什么色差",
            "秤盘不平，一边高一边低。这个价位不应该这样了。按键硬，按起来费力。底座盘不平，有一个脚悬空",
            "底座四个点不平，质量非常差，本来就是有精度要求，结果买一个底座不平的电子秤。简直是一个笑话",
            "f2@#$%#@@(*(*#@(*Y*(H*((@*#(Hskhkashkj欧)))))))",
            "? TP-LINK TL-WR886N 450M无线路由器（宝蓝） WIFI无线穿墙王",
            "好好好好好好好好好好好好好好好",
            "T收到东西很快，包装精美，京东618真的很便宜哦！",
            "T给老爸买来用的，听听音乐，看看文件，上个网足够了",
            "T凑单买的，京东物流就是快上午下单，下午就到了",
            "T苹果味道挺不错水分也足还甜就是有几个伤到骨头好像开始烂了",
            "T用了不到三个星期，电话打不出去，也挂不断，找客服，让我去维修点检测，我一看发来的维修点还在省会，不能退不能换，让我自己不上班出车费去吗？这是老子第一次用华为也是最后一次！",
            "T中粮董事长：蒙牛若再现质量问题市场将不会饶恕 全文: http:t.cn/zj2mwFp （分享自 @Wind资讯）",
            "V裤子质量不错非常好，面料也厚实，尺码很合身",
            "V挺好，质量不错，舒服，值得买",
            "V对于大部分不抽烟，不喝蒙牛的朋友，这条的启示是别吃隔夜菜。",
            "V裤子不是很好，面料也没弹性，版型也不怎么好，和图片也是两回事",
            "V根本不是纯棉，一颗星都不想给",
            "V谬论？内蒙基层官员：蒙牛伊利没任何毛病 源头不安全 http:t.cn/zjzVHW4 （分享自 @新浪财经）",]

key_to_index = word_embedding.key_to_index
tokens = dataset.tokenization(contents, stopwords)
tokens = [[word for word in token if word in key_to_index] for token in tokens]
tokens_truncated = [token[:config.model_sequence_length] for token in tokens]
tokens_padded = [dataset.padding(token, config.model_sequence_length) for token in tokens_truncated]
tokens_ids = [[key_to_index[word] for word in token if word in key_to_index] for
              token in tokens_padded]

for i, token_id in enumerate(tokens_ids):
    data = torch.LongTensor(numpy.array(token_id)).unsqueeze(0).to(config.device)
    output = model(data)
    predicted = output.argmax(dim=1).cpu().numpy()
    print(contents[i], predicted[0], dataset.label2emotion.get(predicted[0]))

test_model[pt/model_Transformer_512_0.4_ReLU_100_35_256_1_0.001_0.8_pt]
word_embedding[pt/embedding.pt]
感觉这个秤非常小巧精致实用。自从看到家人买来这个秤以后，其他的包括秤（包括金器秤）都被闲置了 0 非常满意
第三次购买了，还算挺好的，希望能更耐用一些。 1 满意
没什想说的，中规中矩，没什么色差 3 不满意
秤盘不平，一边高一边低。这个价位不应该这样了。按键硬，按起来费力。底座盘不平，有一个脚悬空 3 不满意
底座四个点不平，质量非常差，本来就是有精度要求，结果买一个底座不平的电子秤。简直是一个笑话 4 非常不满意
f2@#$%#@@(*(*#@(*Y*(H*((@*#(Hskhkashkj欧))))))) 4 非常不满意
? TP-LINK TL-WR886N 450M无线路由器（宝蓝） WIFI无线穿墙王 2 中立
好好好好好好好好好好好好好好好 0 非常满意
T收到东西很快，包装精美，京东618真的很便宜哦！ 0 非常满意
T给老爸买来用的，听听音乐，看看文件，上个网足够了 2 中立
T凑单买的，京东物流就是快上午下单，下午就到了 0 非常满意
T苹果味道挺不错水分也足还甜就是有几个伤到骨头好像开始烂了 3 不满意
T用了不到三个星期，电话打不出去，也挂不断，找客服，让我去维修点检测，我一看发来的维修点还在省会，不能退不能换，让我自己不上班出车费去吗？这是老子第一次用华为也是最后一次！ 4 非常不满意
T中粮董事长：蒙牛若再现质量问题市场将不会饶恕 全文: http:t.cn/zj2mwFp （分享自 @Wind资讯） 2 中立
V裤子质量不错非常好，面料也厚实，尺码很合身 0 非常满意
V挺好，质量不错，舒服，值得买 1 满意
V对于大部分不抽烟，不喝蒙牛的朋友，这条的启示是别吃隔夜菜。 4 非常不满意
V裤子不是很好，面料也没弹性，版型也不怎么好，和图片也是两回事 3 不满意
V根本不是纯棉，一颗星都不想给 4 非常不满意
V谬论？内蒙基层官员：蒙牛伊利没任何毛病 源头不安全 http:t.cn/zjzVHW4 （分享自 @新浪财经） 2 中立
